<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">ML Capstone Project</font></h1>

So in this capstone project, the goal is to be creative
and come up with my own idea or problem to solve using location data.
For example, one can choose to 
- compare different neighborhoods in terms of a service.
- Search for potential explanation of why a neighborhood is popular.
- The cause of complaints in another neighborhood, or
- anything else related to neighborhoods.
Hence the name of the capstone project will be the 'Battle of the Neighborhoods'. 

In [1]:
import geocoder
from bs4 import BeautifulSoup
import requests
import matplotlib
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
import bs4
print(bs4.__version__)


4.9.3


In [3]:
#!conda install -c conda-forge geopy --yes
#!conda uninstall -c conda-forge geopy --yes

## 1. Scrape and Create the Dataframe
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 
    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


### To scrape the data from the website, use the library 'requests' to open and return the html.  Then parse it using the beautifulsoup library.  Put it in a pandas dataframe & clean it & define its shape.


In [4]:
# get the information about Post Code of the neighborhoods of Toronto from wikipage and clean it
#Use function “prettify” to look at nested structure of HTML page
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')
#print(soup.prettify())


In [5]:
#Extract the First Table using the python 'find' command; display the table
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [6]:
#Extract the Postal Codes from the HTML table
#First find all the Table Rows i.e. all the rows with data
table_rows = table.find_all('tr')
data = []  #Initialize the list to contain the rows
for row in table_rows:   #Loop through each row in the table rows list
    td=[]  #Initialize the list to contain each individual row
    for t in row.find_all('td'):  #Loop through each element in the list of data in a given row
        td.append(t.text.strip())  #append each element in the list: td.
    data.append(td) #gather each list 'td' into the 'data' list
    
#the pd.set_option simply allows us to see all the data in the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(data, columns=['Postal_Code', 'Borough', 'Neighborhood'])  #Convert the list data into a Panda Dataframe

In [13]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)

#Below, the groupby is applied to all three columns.  So effectively, each column is deduped within itself and 
#then the last column (Neighborhood), the unique values are joined / concatenated with ',' and the unique values.
df = df.groupby(['Postal_Code','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()

In [14]:
df.shape

(103, 3)

### Get the Latitude and Longitude of the Coordinates for each Postal Code - Merge them to the Pandas DF created above

### geocoder Failed to pull in any data so went to .csv

In [15]:
df1 = pd.read_csv("C:\\Users\\StephenVoorhees\\Documents\\_Training\\Python\\Projects\\ML-Capstone2\\Battle_of_Neighborhoods\\Geospatial_Coordinates.csv") 
df2= pd.merge(df,df1, how= 'inner', on='Postal_Code')
df2.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
